<a href="https://colab.research.google.com/github/DS-Dalen22/DedSec-Dalen/blob/main/Final_Notebook_CV_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Дипломный проект - финальный ноутбук

* Импорт необходимых библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install ultralytics
import os
import cv2
import requests
from google.colab import files
from ultralytics import YOLO
from IPython.display import Video as show_video
from IPython.display import HTML

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.1 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


* Код собран в одну главную функцию, требуется лишь запуск ячейки и загрузка видео ниже
* После обработанное видео автоматически начинает скачивание локально

**Код разбит на функции**

* upload_file() – загрузка данных

* preprocess_video() – предобработка

* load_model() – загрузка модели

* predict_video() – инференс + постобработка

* show_result() – вывод результата

* main_pipeline() - общий запуск

Каждая функция использует только свои входные аргументы

Глобальные переменные не используются

In [ ]:
def load_model(weights_path="DetectModelS3best.pt"):

    try:
        model = YOLO(weights_path)
        print(f"Модель загружена: {weights_path}")
        return model
    except Exception as e:
        raise RuntimeError(f"Ошибка {weights_path}: {e}")


def upload_file(default_path="input.mp4"):
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        return filename
    return default_path


def preprocess_video(input_path, output_size=(960, 960)):
    return input_path


def predict_video(model, input_path, output_path="output.mp4", conf=0.5):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        results = model(frame, conf=conf)
        count = sum(1 for box in results[0].boxes if int(box.cls[0]) == 0)
        annotated_frame = results[0].plot()
        cv2.putText(annotated_frame, f"People: {count}",
                    (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2)
        out.write(annotated_frame)

    cap.release()
    out.release()
    return output_path


def show_result(output_path="output.mp4"):
    return HTML(f"""
    <video width="720" height="480" controls>
        <source src="{output_path}" type="video/mp4">
    </video>
    """)


def main_pipeline(weights_path, conf=0.5):
    input_path = upload_file()
    clean_path = preprocess_video(input_path)
    model = load_model(weights_path)
    processed_video = predict_video(model, clean_path, conf=conf)

    show_result(processed_video)

    files.download(processed_video)

WEIGHTS_PATH = "/content/drive/MyDrive/FinalProject/model/finalMedium_best.pt"

main_pipeline(WEIGHTS_PATH)

Saving people.mp4 to people (1).mp4
✅ Модель загружена: /content/drive/MyDrive/FinalProject/model/finalMedium_best.pt

0: 384x640 20 persons, 25.7ms
Speed: 1.7ms preprocess, 25.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 25.1ms
Speed: 2.0ms preprocess, 25.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 25.0ms
Speed: 2.1ms preprocess, 25.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 20.2ms
Speed: 2.0ms preprocess, 20.2ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 18.2ms
Speed: 2.4ms preprocess, 18.2ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 18.2ms
Speed: 2.0ms preprocess, 18.2ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 persons, 18.1ms
Speed: 2.2ms preprocess, 18.1ms inference, 2.3ms postprocess per image at shape (1,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>